In [2]:

import sys
import os
current_dir = os.path.dirname(os.path.abspath('./'))
if not current_dir in sys.path:
    sys.path.append(current_dir)
from combiners import EpochsCombiner
from typing import *
import mne
import tensorflow as tf
import mneflow as mf
import matplotlib.pyplot as plt
import numpy as np


2022-02-20 21:01:11.758741: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-20 21:01:11.758790: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
mne.set_log_level('CRITICAL')

In [6]:
content_root = '../'
subjects_folder_path = os.path.join(content_root, 'Source/Subjects')
subject_path = os.path.join(subjects_folder_path, 'Ga_Fed_06')
info_path = os.path.join(subject_path, 'Info',
                        'ML_Subject05_P1_tsss_mc_trans_info.pkl')
resp_lock_lm_B1_epochs_path = os.path.join(
    subject_path, 'Epochs', 'RespCor_LM_B1_epochs.fif')
resp_lock_rm_B1_epochs_path = os.path.join(
    subject_path, 'Epochs', 'RespCor_RM_B1_epochs.fif')
resp_lock_li_B1_epochs_path = os.path.join(
    subject_path, 'Epochs', 'RespCor_LI_B1_epochs.fif')
resp_lock_ri_B1_epochs_path = os.path.join(
    subject_path, 'Epochs', 'RespCor_RI_B1_epochs.fif')

In [4]:
resp_lock_lm_B1_epochs = mne.read_epochs(resp_lock_lm_B1_epochs_path)
resp_lock_li_B1_epochs = mne.read_epochs(resp_lock_li_B1_epochs_path)
resp_lock_rm_B1_epochs = mne.read_epochs(resp_lock_rm_B1_epochs_path)
resp_lock_ri_B1_epochs = mne.read_epochs(resp_lock_ri_B1_epochs_path)

resp_lock_li_B1_epochs.resample(200)
resp_lock_lm_B1_epochs.resample(200)
resp_lock_ri_B1_epochs.resample(200)
resp_lock_rm_B1_epochs.resample(200)

Number of events,60
Events,0: 60
Time range,-0.500 – 0.495 sec
Baseline,off


In [5]:
combiner = EpochsCombiner(
    resp_lock_lm_B1_epochs.copy(),
    resp_lock_li_B1_epochs.copy(),
    resp_lock_rm_B1_epochs.copy(),
    resp_lock_ri_B1_epochs.copy()
)
first_class_indices = (0, 1)
second_class_indices = (2, 3)
combiner.combine(first_class_indices, second_class_indices, shuffle=True)

EpochsCombiner([<EpochsFIF |  58 events (all good), -0.5 - 0.495 sec, baseline off, ~23.2 MB, data loaded,
                 '0': 58>,
                <EpochsFIF |  30 events (all good), -0.5 - 0.495 sec, baseline off, ~14.4 MB, data loaded,
                 '0': 30>,
                <EpochsFIF |  60 events (all good), -0.5 - 0.495 sec, baseline off, ~23.8 MB, data loaded,
                 '0': 60>,
                <EpochsFIF |  60 events (all good), -0.5 - 0.495 sec, baseline off, ~23.8 MB, data loaded,
                 '0': 60>])

In [12]:

savepath = '../Source/Subjects/Az_Mar_05/TFR/left_vs_right/'
project_name = 'fake_name'

import_opt = dict(
        savepath=savepath+'/',
        out_name=project_name,
        fs=200,
        input_type='trials',
        target_type='int',
        picks={'meg': 'grad'},
        scale=True,
        crop_baseline=True,
        decimate=None,
        scale_interval=(0, 60),
        n_folds=5,
        overwrite=True,
        segment=False,
        test_set='holdout'
    )
meta = mf.produce_tfrecords((combiner.X, combiner.Y), **import_opt)
dataset = mf.Dataset(meta, train_batch=100)
lf_params = dict(
        n_latent=32,
        filter_length=50,
        nonlin=tf.keras.activations.elu,
        padding='SAME',
        pooling=10,
        stride=10,
        pool_type='max',
        model_path=import_opt['savepath'],
        dropout=.4,
        l2_scope=["weights"],
        l2=1e-6
)

model = mf.models.LFCNN(dataset, lf_params)
model.build()

importing from tuple
input shapes: X- (208, 204, 200) targets- (208, 1)
Preprocessing:
Scaling to interval 0.0 - 60.0
Splitting sets
Preprocessed: (208, 1, 200, 204) (208, 2) folds: 6 x 34
Prepocessed sample shape: (1, 200, 204)
Target shape actual/metadata:  (2,) (2,)
Saving TFRecord# 0
Setting reg for dmx, to l2
Built: dmx input: (None, 1, 200, 204)
Setting reg for tconv, to l2
Built: tconv input: (None, 1, 200, 32)
Setting reg for fc, to l1
Built: fc input: (None, 1, 20, 32)
Input shape: (1, 200, 204)
y_pred: (None, 2)
Initialization complete!


In [13]:
weights_path = os.path.join(subject_path, 'Weights', 'LM&LI_vs_RM&RI_B1-B8.h5')
model.km.load_weights(weights_path)

In [14]:
X = combiner.X.copy()
print(X.shape)
X = np.transpose(np.expand_dims(X, axis = 1), (0, 1, 3, 2))
print(X.shape)

(208, 204, 200)
(208, 1, 200, 204)


In [15]:
model.km(X)

<tf.Tensor: shape=(208, 2), dtype=float32, numpy=
array([[-0.00817292,  0.07730297],
       [-0.00817292,  0.07730297],
       [-0.00817292,  0.07730297],
       [-0.00817292,  0.07730297],
       [-0.00817292,  0.07730297],
       [-0.00817292,  0.07730297],
       [-0.00817292,  0.07730297],
       [-0.00817292,  0.07730297],
       [-0.00817292,  0.07730297],
       [-0.00817292,  0.07730297],
       [-0.00817292,  0.07730297],
       [-0.00817292,  0.07730297],
       [-0.00817292,  0.07730297],
       [-0.00817292,  0.07730297],
       [-0.00817292,  0.07730297],
       [-0.00817292,  0.07730297],
       [-0.00817292,  0.07730297],
       [-0.00817292,  0.07730297],
       [-0.00817292,  0.07730297],
       [-0.00817292,  0.07730297],
       [-0.00817292,  0.07730297],
       [-0.00817292,  0.07730297],
       [-0.00817292,  0.07730297],
       [-0.00817292,  0.07730297],
       [-0.00817292,  0.07730297],
       [-0.00817292,  0.07730297],
       [-0.00817292,  0.07730297],
     

In [10]:
models_path = '/home/user/Projects/FingerMovementDecoder/Source/Subjects/Ga_Fed_06/Models'
model = tf.keras.models.load_model(
    os.path.join(models_path, 'LM&LI_vs_RM&RI_B1-B8.h5'),
    custom_objects={
        'DeMixing': mf.layers.DeMixing,
        'LFTConv': mf.layers.LFTConv,
        'TempPooling': mf.layers.TempPooling,
        'Dense': mf.layers.Dense,
        'identity': tf.identity,
        'softmax_cross_entropy_with_logits_v2': tf.compat.v1.nn.softmax_cross_entropy_with_logits_v2,
    }
)

Built: dmx input: (None, 1, 200, 204)
Built: tconv input: (None, 1, 200, 32)
Built: fc input: (None, 1, 20, 32)


In [11]:
model(X)

<tf.Tensor: shape=(208, 2), dtype=float32, numpy=
array([[-0.00817292,  0.07730297],
       [-0.00817292,  0.07730297],
       [-0.00817292,  0.07730297],
       [-0.00817292,  0.07730297],
       [-0.00817292,  0.07730297],
       [-0.00817292,  0.07730297],
       [-0.00817292,  0.07730297],
       [-0.00817292,  0.07730297],
       [-0.00817292,  0.07730297],
       [-0.00817292,  0.07730297],
       [-0.00817292,  0.07730297],
       [-0.00817292,  0.07730297],
       [-0.00817292,  0.07730297],
       [-0.00817292,  0.07730297],
       [-0.00817292,  0.07730297],
       [-0.00817292,  0.07730297],
       [-0.00817292,  0.07730297],
       [-0.00817292,  0.07730297],
       [-0.00817292,  0.07730297],
       [-0.00817292,  0.07730297],
       [-0.00817292,  0.07730297],
       [-0.00817292,  0.07730297],
       [-0.00817292,  0.07730297],
       [-0.00817292,  0.07730297],
       [-0.00817292,  0.07730297],
       [-0.00817292,  0.07730297],
       [-0.00817292,  0.07730297],
     

In [7]:
tfr_path = os.path.join(subject_path, 'TFR', 'LM&LI_vs_RM&RI_B1-B8', 'fingers_movement_epochs_test_0.tfrecord')

raw_dataset = tf.data.TFRecordDataset([tfr_path])
raw_dataset

2022-02-20 21:02:07.521458: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-02-20 21:02:07.521505: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-02-20 21:02:07.521531: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (arcolinux-machine): /proc/driver/nvidia/version does not exist
2022-02-20 21:02:07.521929: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


<TFRecordDatasetV2 element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>

In [8]:
for element in raw_dataset:
      print(element)

